# Test Selection Analysis

In [251]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from scipy.spatial import distance

In [252]:
# base_dir = "trained_models/"
# base_dir = "trained_models/backup/"
# base_dir = "trained_models/backup_10epochs/"
# base_dir = "trained_models/backup_20epochs_50sample/"
base_dir = "trained_models/backup_30epochs_50sample/"
def load_data(train, test, fname) :
    fpath = base_dir + "resnet18_{}_piecewise_eps8_bs128_maxlr0.1_ls0.3_BNeval/eval/{}/{}.txt".format(train, test, fname)
#     if train == "all" or train == "combine_autopgd_pixelattack_spatialtransformation_squareattack" or train == "combine_newtonfool_pixelattack_spatialtransformation_squareattack":
#             fpath = "trained_models/resnet18_{}_piecewise_eps8_bs128_maxlr0.1_ls0.3_BNeval/eval/{}/{}.txt".format(train, test, fname)
    return np.loadtxt(fpath).astype(np.int)

train = "autoattack"
# train = "combine_pixelattack_spatialtransformation_autopgd"
# train = "combine_pixelattack_spatialtransformation_autopgd_autoattack"
# train = "combine_cw_bim_pixelattack_spatialtransformation"
test = "pgd"

y_original = load_data(train, test, "Y_original")
y_original_pred = load_data(train, test, "Y_original_pred")
y_adv = load_data(train, test, "Y_adv")
y_adv_pred = load_data(train, test, "Y_adv_pred")

In [253]:
def get_robust_data(train, test):
    y_adv = load_data(train, test, "Y_adv")
    y_adv_pred = load_data(train, test, "Y_adv_pred")
    return y_adv, y_adv_pred

# train = "pgd"
# train = "combine_cw_bim_pixelattack_spatialtransformation"
# train = "combine_autoattack_autopgd_pixelattack_spatialtransformation"
# train = "combine_pixelattack_spatialtransformation_autoattack_autopgd"
test = "autoattack"
y_adv, y_adv_pred = get_robust_data(train, test)

print("Y_adv({},{}): {}".format(train, test, y_adv))
print("Y_adv_pred({},{}): {}".format(train, test, y_adv_pred))

Y_adv(autoattack,autoattack): [3 8 8 ... 5 1 7]
Y_adv_pred(autoattack,autoattack): [3 8 8 ... 5 2 7]


In [254]:
attacks = ["autoattack", "autopgd", "bim", "cw", "fgsm", "pgd", "deepfool", "newtonfool", "squareattack", "pixelattack", "spatialtransformation"]

In [258]:
# train = attacks.copy()
# train.append("all")


train = []

### 10 epochs, 100% data
# train.append("combine_pixelattack_spatialtransformation_cw_autopgd")
# train.append("combine_pixelattack_spatialtransformation_autopgd")

### 20 epochs, 50% sample
# train.append("combine_pixelattack_spatialtransformation_autoattack_autopgd")
# train.append("combine_pixelattack_spatialtransformation_autopgd_autoattack")

### 30 epochs, 50% sample
train.append("combine_cw_bim_pixelattack_spatialtransformation")
# train.append("combine_bim_cw_pixelattack_spatialtransformation")
# train.append("combine_cw_pixelattack_spatialtransformation_bim")
train.append("combine_pixelattack_spatialtransformation_cw_bim")

test = attacks.copy()
test.append("all")

In [259]:
predictions = {}
labels = {}

for a1 in train :
    p = np.array([])
    l = np.array([])
    for a2 in attacks :
        y, y_pred = get_robust_data(a1, a2)
        p = np.concatenate((p, y_pred))
        l = np.concatenate((l, y))
    predictions[a1] = {}
    labels[a1] = {}
    predictions[a1]["all"] = p
    labels[a1]["all"] = l

for a1 in train :
    for a2 in attacks :
        y, y_pred = get_robust_data(a1, a2)
        predictions[a1][a2] = y_pred
        labels[a1][a2] = y

In [260]:
# predictions

In [261]:
# labels

In [262]:
accs = {}
for tr in train :
    accs[tr] = {}
    for ts in test :
        accs[tr][ts] = (labels[tr][ts] == predictions[tr][ts]).sum().item() / len(labels[tr][ts])

    
accs = pd.DataFrame(data=accs).T
accs

,autoattack,autopgd,bim,cw,fgsm,pgd,deepfool,newtonfool,squareattack,pixelattack,spatialtransformation,all
combine_cw_bim_pixelattack_spatialtransformation,0.5788,0.5806,0.5748,0.6067,0.5608,0.5737,0.5794,0.6079,0.6136,0.1611,0.7667,0.564009
combine_pixelattack_spatialtransformation_cw_bim,0.7368,0.7381,0.7344,0.7525,0.7122,0.7335,0.7387,0.7498,0.7441,0.1721,0.4876,0.663618


In [263]:
sorted_acc = accs.sort_values(by="all", ascending=False)
sorted_acc

,autoattack,autopgd,bim,cw,fgsm,pgd,deepfool,newtonfool,squareattack,pixelattack,spatialtransformation,all
combine_pixelattack_spatialtransformation_cw_bim,0.7368,0.7381,0.7344,0.7525,0.7122,0.7335,0.7387,0.7498,0.7441,0.1721,0.4876,0.663618
combine_cw_bim_pixelattack_spatialtransformation,0.5788,0.5806,0.5748,0.6067,0.5608,0.5737,0.5794,0.6079,0.6136,0.1611,0.7667,0.564009


## Time Execution Measurement

In [264]:
import datetime

def get_adv_train_time_execution(train) :
#     fpath = "trained_models/{}/output.log".format(train)
    fpath = base_dir + "resnet18_{}_piecewise_eps8_bs128_maxlr0.1_ls0.3_BNeval/output.log".format(train)
#     if train == "all" or train == "combine_autopgd_pixelattack_spatialtransformation_squareattack" or train == "combine_newtonfool_pixelattack_spatialtransformation_squareattack":
#             fpath = "trained_models/resnet18_{}_piecewise_eps8_bs128_maxlr0.1_ls0.3_BNeval/output.log".format(train)

    
    file = open(fpath)
    lines = file.readlines()
    file.close()
    
    start_time = None
    end_time = None
    
    datefmt='%Y/%m/%d %H:%M:%S'
    
    
    for l in lines :
        token = l.split("-")
        date_str = token[0][1:-2]
        date_time = datetime.datetime.strptime(date_str, datefmt)
        if token[1][1:11] == "Namespace(" :
            start_time = date_time
        else :
            end_time = date_time
    
    if end_time > start_time :
        return end_time-start_time
    raise ValueError("Duration not found")
    
# _train = "autoattack"
# _train = "all"
# get_adv_train_time_execution(_train) + get_adv_train_time_execution("fgsm") 

In [265]:
durations = []
for tr in train :
    if tr[:7] == "combine" :
        duration = None
        for i in range(len(attacks)) :
            a = attacks[i]
            if i == 0 :
                duration = get_adv_train_time_execution(a)
            else :
                duration += get_adv_train_time_execution(a)
        duration+= get_adv_train_time_execution(tr)
        durations.append(duration)
    else :
        durations.append(get_adv_train_time_execution(tr))

accs["duration"] = durations

In [266]:
accs["all"]
# accs

combine_cw_bim_pixelattack_spatialtransformation    0.564009
combine_pixelattack_spatialtransformation_cw_bim    0.663618
Name: all, dtype: float64

In [267]:
sorted_acc = accs.sort_values(by="all", ascending=False)
sorted_acc

,autoattack,autopgd,bim,cw,fgsm,pgd,deepfool,newtonfool,squareattack,pixelattack,spatialtransformation,all,duration
combine_pixelattack_spatialtransformation_cw_bim,0.7368,0.7381,0.7344,0.7525,0.7122,0.7335,0.7387,0.7498,0.7441,0.1721,0.4876,0.663618,0 days 16:46:21
combine_cw_bim_pixelattack_spatialtransformation,0.5788,0.5806,0.5748,0.6067,0.5608,0.5737,0.5794,0.6079,0.6136,0.1611,0.7667,0.564009,0 days 16:47:25


In [268]:
selected_sorted_acc = sorted_acc[:][["all", "duration"]]
selected_sorted_acc.rename(columns={"all":"test_on_all_adv"})

,test_on_all_adv,duration
combine_pixelattack_spatialtransformation_cw_bim,0.663618,0 days 16:46:21
combine_cw_bim_pixelattack_spatialtransformation,0.564009,0 days 16:47:25
